In [1]:
import pandas as pd
import pandasql as psql
import pickle
import os

In [9]:
import pandas as pd
import pandasql as psql
import os

target_patients = pd.read_csv('/media/data/huyennm/mimic-iv/notebook/eligible_patients.csv')
patients_df = pd.read_csv('/media/data3/biodataset/MIMIC_IV/MIMIC-IV-v2.2/physionet.org/files/mimiciv/2.2/hosp/patients.csv')
height_df = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/height.csv')
weight_df = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/weight.csv')
admissions_df = pd.read_csv('/media/data3/biodataset/MIMIC_IV/MIMIC-IV-v2.2/physionet.org/files/mimiciv/2.2/hosp/admissions.csv')
omr = pd.read_csv('/media/data3/biodataset/MIMIC_IV/MIMIC-IV-v2.2/physionet.org/files/mimiciv/2.2/hosp/omr.csv')
gcs_df = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/gcs.csv')
dur_mv = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/duration_mv.csv')

/tmp/ipykernel_1693931/3812884214.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  weight_df = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/weight.csv')


In [2]:
procedureevents_mv = pd.read_csv('/media/data3/biodataset/MIMIC_IV/MIMIC-IV-v2.2/physionet.org/files/mimiciv/2.2/icu/procedureevents.csv')
procedureevents_mv.head()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10000032,29079034,39553978,88981.0,2180-07-23 14:43:00,2180-07-23 14:44:00,2180-07-23 14:43:00,225966,1.0,NaN,...,6416557,6416557,Procedures,Task,39.4,0,0,FinishedRunning,1.0,0
1,10000032,29079034,39553978,NaN,2180-07-23 14:24:00,2180-07-23 23:50:00,2180-07-23 23:50:49.983,224275,566.0,min,...,6497934,6497934,Peripheral Lines,ContinuousProcess,39.4,1,0,FinishedRunning,566.0,1
2,10000032,29079034,39553978,NaN,2180-07-23 14:24:00,2180-07-23 23:50:00,2180-07-23 23:50:49.983,224277,566.0,min,...,9643097,9643097,Peripheral Lines,ContinuousProcess,39.4,1,0,FinishedRunning,566.0,1
3,10000980,26913865,39765666,NaN,2189-06-27 09:01:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,225794,697.0,min,...,5989583,5989583,Ventilation,ContinuousProcess,76.2,1,0,FinishedRunning,697.0,1
4,10000980,26913865,39765666,NaN,2189-06-27 09:15:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,224277,683.0,min,...,476764,476764,Peripheral Lines,ContinuousProcess,76.2,1,0,FinishedRunning,683.0,1


In [4]:
bmi_omr = omr[(omr['result_name'] == 'BMI (kg/m2)') & (omr['subject_id'].isin(target_patients['subject_id']))]
bmi_omr

,subject_id,chartdate,seq_num,result_name,result_value
329,10000980,2185-06-17,1,BMI (kg/m2),33.6
331,10000980,2185-09-17,1,BMI (kg/m2),34.0
333,10000980,2185-12-17,1,BMI (kg/m2),34.2
336,10000980,2186-03-17,1,BMI (kg/m2),34.0
340,10000980,2186-09-15,1,BMI (kg/m2),34.2
...,...,...,...,...,...
6438580,19998330,2178-06-20,1,BMI (kg/m2),23.9
6438585,19998330,2178-07-10,1,BMI (kg/m2),24.6
6438588,19998330,2178-08-08,1,BMI (kg/m2),24.7
6438597,19998330,2178-11-11,1,BMI (kg/m2),25.0


In [29]:
bmi_omr.dtypes

subject_id       int64
chartdate       object
seq_num          int64
result_name     object
result_value    object
dtype: object

In [27]:
bmi_omr[bmi_omr['subject_id'] == 19998330]

,subject_id,chartdate,seq_num,result_name,result_value
6438573,19998330,2178-02-21,1,BMI (kg/m2),37.2
6438577,19998330,2178-05-21,1,BMI (kg/m2),35.5
6438580,19998330,2178-06-20,1,BMI (kg/m2),23.9
6438585,19998330,2178-07-10,1,BMI (kg/m2),24.6
6438588,19998330,2178-08-08,1,BMI (kg/m2),24.7
6438597,19998330,2178-11-11,1,BMI (kg/m2),25.0
6438601,19998330,2178-12-03,1,BMI (kg/m2),26.4


In [5]:
target_patients

,subject_id,hadm_id,stay_id,charttime,vent,age_at_admission
0,10001843,26133978,39698942,2134-12-05 19:00:00,1,76
1,10001843,26133978,39698942,2134-12-05 20:00:00,1,76
2,10001843,26133978,39698942,2134-12-05 21:00:00,1,76
3,10001843,26133978,39698942,2134-12-05 21:01:00,1,76
4,10001843,26133978,39698942,2134-12-05 23:00:00,1,76
...,...,...,...,...,...,...
201344,19998591,24349193,31144045,2185-07-17 16:00:00,1,53
201345,19998591,24349193,36794489,2185-07-04 07:50:00,1,53
201346,19998591,24349193,36794489,2185-07-04 12:00:00,1,53
201347,19998591,24349193,36794489,2185-07-04 16:00:00,1,53


In [4]:
gender_query = """
SELECT
    tp.*,
    p.gender
FROM target_patients tp
LEFT JOIN patients_df p ON tp.subject_id = p.subject_id;
"""

In [5]:
def add_gender(target_patients, patients_df):
    return psql.sqldf(gender_query, locals())

In [6]:
race_query = """
SELECT
    tp.*,
    a.race
FROM target_patients tp
LEFT JOIN admissions_df a ON tp.subject_id = a.subject_id AND tp.hadm_id = a.hadm_id;
"""

In [7]:
def add_race(target_patients, admissions_df):
    return psql.sqldf(race_query, locals())

In [8]:
height_query = """
WITH height_data AS (
    SELECT
        ht.subject_id,
        ht.hadm_id,
        ht.stay_id,
        ht.charttime,
        CASE 
            WHEN ht.itemid = 226707 THEN ht.valuenum * 0.0254  -- Convert inches to meters
            ELSE ht.valuenum / 100  -- Convert cm to meters
        END AS height_value  -- Rename the height column to avoid ambiguity
    FROM height_df ht
)
SELECT
    tp.*,  -- Select all columns from target_patients
    ROUND(AVG(
        CASE 
            -- Priority 1: stay_id match within time range
            WHEN ht.stay_id = tp.stay_id 
                 AND ht.charttime >= DATETIME(tp.intime, '-6 hours')
                 AND ht.charttime <= DATETIME(tp.intime, '+1 day')
            THEN height_value

            -- Priority 2: hadm_id match
            WHEN ht.hadm_id = tp.hadm_id
            THEN height_value

            -- Priority 3: subject_id match
            WHEN ht.subject_id = tp.subject_id 
            THEN height_value
            ELSE NULL
        END
    ), 2) AS height  -- The final height column to be used in the result
FROM target_patients tp
LEFT JOIN height_data ht
    ON (tp.stay_id = ht.stay_id
        AND ht.charttime >= DATETIME(tp.intime, '-6 hours')
        AND ht.charttime <= DATETIME(tp.intime, '+1 day'))
    OR tp.hadm_id = ht.hadm_id
    OR tp.subject_id = ht.subject_id
GROUP BY tp.subject_id, tp.stay_id;
"""

In [9]:
def add_height(target_patients, height_df):
    height_df['charttime'] = pd.to_datetime(height_df['charttime'])
    return psql.sqldf(height_query, locals())

In [10]:
weight_query = """
SELECT
    tp.*,
    AVG(
        CASE 
            WHEN wt.itemid = 226512 THEN wt.valuenum  -- Weight in kilograms
            WHEN wt.itemid = 226531 THEN wt.valuenum / 2.2046  -- Convert pounds to kilograms
            ELSE NULL 
        END
    ) AS weight_admit,
    AVG(
        CASE 
            WHEN wt.itemid = 224639 THEN wt.valuenum -- Weight on first ICU day
            ELSE NULL
        END
    ) AS weight_first_day
FROM target_patients tp
LEFT JOIN weight_df wt
ON tp.stay_id = wt.stay_id AND wt.charttime <= DATETIME(tp.intime, '+1 day')
GROUP BY tp.subject_id, tp.stay_id
"""

In [11]:
def add_weight(target_patients, weight_df):
    weight_df['charttime'] = pd.to_datetime(weight_df['charttime'])
    return psql.sqldf(weight_query, locals())

In [12]:
target_patients = add_gender(target_patients, patients_df)
target_patients

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender
0,17656146,20003587,34504641,2146-04-17 08:32:01.000000,2146-04-18 05:05:23,0.856505,59,F
1,19657904,20004357,30729609,2157-08-05 12:22:00.000000,2157-08-08 20:03:44,3.320648,77,F
2,17368229,20006154,30306704,2139-03-17 23:00:00.000000,2139-03-21 16:31:49,3.730428,71,F
3,17844820,20014219,38957578,2142-10-06 13:38:14.000000,2142-10-25 16:50:43,19.133669,70,M
4,13399796,20020590,39167908,2121-11-17 20:18:32.000000,2121-11-20 03:20:40,2.293148,76,F
...,...,...,...,...,...,...,...,...
4775,10004401,29988601,32773003,2144-01-26 22:28:04.000000,2144-02-06 13:44:15,10.636238,85,M
4776,17315464,29991213,30757265,2120-09-07 18:48:44.000000,2120-09-09 18:29:51,1.986887,85,M
4777,11165060,29991539,36423523,2137-11-21 00:14:31.000000,2137-12-13 18:15:08,22.750428,69,M
4778,11282384,29994261,30592303,2124-09-16 08:13:00.000000,2124-09-21 18:35:27,5.432257,69,F


In [13]:
target_patients = add_race(target_patients, admissions_df)
target_patients

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race
0,17656146,20003587,34504641,2146-04-17 08:32:01.000000,2146-04-18 05:05:23,0.856505,59,F,BLACK/AFRICAN AMERICAN
1,19657904,20004357,30729609,2157-08-05 12:22:00.000000,2157-08-08 20:03:44,3.320648,77,F,BLACK/AFRICAN AMERICAN
2,17368229,20006154,30306704,2139-03-17 23:00:00.000000,2139-03-21 16:31:49,3.730428,71,F,UNKNOWN
3,17844820,20014219,38957578,2142-10-06 13:38:14.000000,2142-10-25 16:50:43,19.133669,70,M,WHITE
4,13399796,20020590,39167908,2121-11-17 20:18:32.000000,2121-11-20 03:20:40,2.293148,76,F,WHITE
...,...,...,...,...,...,...,...,...,...
4775,10004401,29988601,32773003,2144-01-26 22:28:04.000000,2144-02-06 13:44:15,10.636238,85,M,WHITE
4776,17315464,29991213,30757265,2120-09-07 18:48:44.000000,2120-09-09 18:29:51,1.986887,85,M,WHITE
4777,11165060,29991539,36423523,2137-11-21 00:14:31.000000,2137-12-13 18:15:08,22.750428,69,M,WHITE
4778,11282384,29994261,30592303,2124-09-16 08:13:00.000000,2124-09-21 18:35:27,5.432257,69,F,BLACK/AFRICAN AMERICAN


In [14]:
target_patients = add_height(target_patients, height_df)
target_patients

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height
0,10002428,23473524,35479615,2156-05-11 14:49:34.000000,2156-05-22 14:16:46,10.977222,81,F,WHITE,1.50
1,10003400,20214994,32128372,2137-02-25 23:37:19.000000,2137-03-10 21:29:36,12.911308,75,F,BLACK/AFRICAN AMERICAN,1.66
2,10003400,23559586,34577403,2137-08-10 19:54:51.000000,2137-08-13 17:54:54,2.916701,75,F,BLACK/AFRICAN AMERICAN,1.66
3,10004235,24181354,34100191,2196-02-24 17:07:00.000000,2196-02-29 15:58:02,4.952106,47,M,BLACK/CAPE VERDEAN,1.83
4,10004401,23920883,32743332,2144-04-21 20:49:00.000000,2144-05-01 13:53:03,9.711146,85,M,WHITE,1.70
...,...,...,...,...,...,...,...,...,...,...
4775,19997293,26366652,36403582,2123-11-17 05:47:01.000000,2123-11-19 17:08:30,2.473252,76,M,WHITE,1.82
4776,19998330,23151993,31085771,2178-09-20 21:40:00.000000,2178-09-21 18:06:26,0.851690,72,F,BLACK/AFRICAN AMERICAN,NaN
4777,19998330,21135114,31417783,2178-10-21 17:20:00.000000,2178-10-23 17:43:28,2.016296,72,F,BLACK/AFRICAN AMERICAN,NaN
4778,19998330,24492004,32641669,2178-10-01 08:51:00.000000,2178-10-03 23:25:08,2.607037,72,F,BLACK/AFRICAN AMERICAN,NaN


In [15]:
target_patients = add_weight(target_patients, weight_df)
target_patients

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day
0,10002428,23473524,35479615,2156-05-11 14:49:34.000000,2156-05-22 14:16:46,10.977222,81,F,WHITE,1.50,48.400000,49.25
1,10003400,20214994,32128372,2137-02-25 23:37:19.000000,2137-03-10 21:29:36,12.911308,75,F,BLACK/AFRICAN AMERICAN,1.66,92.902976,94.70
2,10003400,23559586,34577403,2137-08-10 19:54:51.000000,2137-08-13 17:54:54,2.916701,75,F,BLACK/AFRICAN AMERICAN,1.66,NaN,90.50
3,10004235,24181354,34100191,2196-02-24 17:07:00.000000,2196-02-29 15:58:02,4.952106,47,M,BLACK/CAPE VERDEAN,1.83,126.823339,130.00
4,10004401,23920883,32743332,2144-04-21 20:49:00.000000,2144-05-01 13:53:03,9.711146,85,M,WHITE,1.70,90.596303,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4775,19997293,26366652,36403582,2123-11-17 05:47:01.000000,2123-11-19 17:08:30,2.473252,76,M,WHITE,1.82,172.756341,NaN
4776,19998330,23151993,31085771,2178-09-20 21:40:00.000000,2178-09-21 18:06:26,0.851690,72,F,BLACK/AFRICAN AMERICAN,NaN,76.205960,NaN
4777,19998330,21135114,31417783,2178-10-21 17:20:00.000000,2178-10-23 17:43:28,2.016296,72,F,BLACK/AFRICAN AMERICAN,NaN,77.514506,77.65
4778,19998330,24492004,32641669,2178-10-01 08:51:00.000000,2178-10-03 23:25:08,2.607037,72,F,BLACK/AFRICAN AMERICAN,NaN,77.100000,74.55


In [17]:
target = target_patients
target

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day
0,10002428,23473524,35479615,2156-05-11 14:49:34.000000,2156-05-22 14:16:46,10.977222,81,F,WHITE,1.50,48.400000,49.25
1,10003400,20214994,32128372,2137-02-25 23:37:19.000000,2137-03-10 21:29:36,12.911308,75,F,BLACK/AFRICAN AMERICAN,1.66,92.902976,94.70
2,10003400,23559586,34577403,2137-08-10 19:54:51.000000,2137-08-13 17:54:54,2.916701,75,F,BLACK/AFRICAN AMERICAN,1.66,NaN,90.50
3,10004235,24181354,34100191,2196-02-24 17:07:00.000000,2196-02-29 15:58:02,4.952106,47,M,BLACK/CAPE VERDEAN,1.83,126.823339,130.00
4,10004401,23920883,32743332,2144-04-21 20:49:00.000000,2144-05-01 13:53:03,9.711146,85,M,WHITE,1.70,90.596303,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4775,19997293,26366652,36403582,2123-11-17 05:47:01.000000,2123-11-19 17:08:30,2.473252,76,M,WHITE,1.82,172.756341,NaN
4776,19998330,23151993,31085771,2178-09-20 21:40:00.000000,2178-09-21 18:06:26,0.851690,72,F,BLACK/AFRICAN AMERICAN,NaN,76.205960,NaN
4777,19998330,21135114,31417783,2178-10-21 17:20:00.000000,2178-10-23 17:43:28,2.016296,72,F,BLACK/AFRICAN AMERICAN,NaN,77.514506,77.65
4778,19998330,24492004,32641669,2178-10-01 08:51:00.000000,2178-10-03 23:25:08,2.607037,72,F,BLACK/AFRICAN AMERICAN,NaN,77.100000,74.55


In [83]:
path_save = './pickle'
os.makedirs(path_save, exist_ok=True)

In [19]:
outfile = open(os.path.join(path_save, 'prepare_bmi.pickle'), 'wb')
pickle.dump(target_patients, outfile)
outfile.close()

In [14]:
path_save = './pickle'
infile = open(os.path.join(path_save,'prepare_bmi.pickle'), 'rb')
new_target = pickle.load(infile)
infile.close()

In [15]:
new_target

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day
0,10002428,23473524,35479615,2156-05-11 14:49:34.000000,2156-05-22 14:16:46,10.977222,81,F,WHITE,1.50,48.400000,49.25
1,10003400,20214994,32128372,2137-02-25 23:37:19.000000,2137-03-10 21:29:36,12.911308,75,F,BLACK/AFRICAN AMERICAN,1.66,92.902976,94.70
2,10003400,23559586,34577403,2137-08-10 19:54:51.000000,2137-08-13 17:54:54,2.916701,75,F,BLACK/AFRICAN AMERICAN,1.66,NaN,90.50
3,10004235,24181354,34100191,2196-02-24 17:07:00.000000,2196-02-29 15:58:02,4.952106,47,M,BLACK/CAPE VERDEAN,1.83,126.823339,130.00
4,10004401,23920883,32743332,2144-04-21 20:49:00.000000,2144-05-01 13:53:03,9.711146,85,M,WHITE,1.70,90.596303,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4775,19997293,26366652,36403582,2123-11-17 05:47:01.000000,2123-11-19 17:08:30,2.473252,76,M,WHITE,1.82,172.756341,NaN
4776,19998330,23151993,31085771,2178-09-20 21:40:00.000000,2178-09-21 18:06:26,0.851690,72,F,BLACK/AFRICAN AMERICAN,NaN,76.205960,NaN
4777,19998330,21135114,31417783,2178-10-21 17:20:00.000000,2178-10-23 17:43:28,2.016296,72,F,BLACK/AFRICAN AMERICAN,NaN,77.514506,77.65
4778,19998330,24492004,32641669,2178-10-01 08:51:00.000000,2178-10-03 23:25:08,2.607037,72,F,BLACK/AFRICAN AMERICAN,NaN,77.100000,74.55


In [14]:
def add_bmi(target_patients):
    target_patients['bmi'] = target_patients.apply(
        lambda row: round(row['weight_admit'] / (row['height'] ** 2), 2) 
        if pd.notnull(row['weight_admit']) and pd.notnull(row['height']) else
        (round(row['weight_first_day'] / (row['height'] ** 2), 2) 
        if pd.notnull(row['weight_first_day']) and pd.notnull(row['height']) else None),
        axis=1
    )
        
    return target_patients

In [75]:
target = add_bmi(new_target)
target[target['bmi'].isna()]

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi
11,10015931,22130791,37093652,2177-03-24 21:48:07.000000,2177-03-29 18:03:36,4.844086,88,M,WHITE,NaN,59.437925,NaN,NaN
30,10048899,20315647,30644814,2165-10-20 19:30:00.000000,2165-10-22 16:23:10,1.870255,91,F,WHITE,NaN,64.836828,NaN,NaN
34,10062020,25255239,35323579,2113-05-31 21:18:47.000000,2113-06-01 15:49:43,0.771481,66,F,WHITE,NaN,99.954710,NaN,NaN
37,10078805,20434240,30950523,2178-07-21 07:47:00.000000,2178-07-22 17:18:24,1.396806,88,M,WHITE,NaN,100.100000,105.00,NaN
43,10094805,21370539,34488364,2182-04-23 19:15:11.000000,2182-04-29 16:59:03,5.905463,74,M,WHITE,NaN,65.835784,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4762,19978239,21674747,38949980,2120-12-02 18:56:20.000000,2120-12-06 21:11:51,4.094109,82,F,WHITE,NaN,101.000000,101.00,NaN
4776,19998330,23151993,31085771,2178-09-20 21:40:00.000000,2178-09-21 18:06:26,0.851690,72,F,BLACK/AFRICAN AMERICAN,NaN,76.205960,NaN,NaN
4777,19998330,21135114,31417783,2178-10-21 17:20:00.000000,2178-10-23 17:43:28,2.016296,72,F,BLACK/AFRICAN AMERICAN,NaN,77.514506,77.65,NaN
4778,19998330,24492004,32641669,2178-10-01 08:51:00.000000,2178-10-03 23:25:08,2.607037,72,F,BLACK/AFRICAN AMERICAN,NaN,77.100000,74.55,NaN


In [32]:
target.dtypes

subject_id            int64
hadm_id               int64
stay_id               int64
intime               object
outtime              object
los                 float64
age_at_admission      int64
gender               object
race                 object
height              float64
weight_admit        float64
weight_first_day    float64
bmi                 float64
dtype: object

In [70]:
# Loop over target where 'bmi' is NaN
for index, row in target[target['bmi'].isna()].iterrows():
    records = bmi_omr[bmi_omr['subject_id'] == row['subject_id']]
    print(f"Found {len(records)} matching records in bmi_omr for subject_id {row['subject_id']}")
        
    if not records.empty:
        records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
        closest_bmi_record = records.loc[records['time_diff'].idxmin()]
        target.at[index, 'bmi'] = closest_bmi_record['result_value']

Found 11 matching records in bmi_omr for subject_id 10015931
Found 0 matching records in bmi_omr for subject_id 10048899
Found 0 matching records in bmi_omr for subject_id 10062020
Found 1 matching records in bmi_omr for subject_id 10078805
Found 0 matching records in bmi_omr for subject_id 10094805
Found 58 matching records in bmi_omr for subject_id 10102822
Found 9 matching records in bmi_omr for subject_id 10104549
Found 0 matching records in bmi_omr for subject_id 10132888
Found 34 matching records in bmi_omr for subject_id 10150056
Found 0 matching records in bmi_omr for subject_id 10155871
Found 0 matching records in bmi_omr for subject_id 10163041
Found 0 matching records in bmi_omr for subject_id 10182211
Found 25 matching records in bmi_omr for subject_id 10221321
Found 0 matching records in bmi_omr for subject_id 10261084
Found 49 matching records in bmi_omr for subject_id 10265482
Found 0 matching records in bmi_omr for subject_id 10282931
Found 1 matching records in bmi_omr

/tmp/ipykernel_1928800/1339896180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/1339896180.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '20.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  target.at[index, 'bmi'] = closest_bmi_record['result_value']
/tmp/ipykernel_1928800/1339896180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

Found 8 matching records in bmi_omr for subject_id 13262421
Found 0 matching records in bmi_omr for subject_id 13266707
Found 0 matching records in bmi_omr for subject_id 13273685
Found 0 matching records in bmi_omr for subject_id 13273685
Found 0 matching records in bmi_omr for subject_id 13281743
Found 0 matching records in bmi_omr for subject_id 13299872
Found 0 matching records in bmi_omr for subject_id 13331522
Found 1 matching records in bmi_omr for subject_id 13364910
Found 65 matching records in bmi_omr for subject_id 13398212
Found 6 matching records in bmi_omr for subject_id 13401524
Found 0 matching records in bmi_omr for subject_id 13421733
Found 0 matching records in bmi_omr for subject_id 13434840
Found 0 matching records in bmi_omr for subject_id 13440565
Found 0 matching records in bmi_omr for subject_id 13444096
Found 0 matching records in bmi_omr for subject_id 13473186
Found 0 matching records in bmi_omr for subject_id 13476942
Found 2 matching records in bmi_omr for

/tmp/ipykernel_1928800/1339896180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/1339896180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/1339896180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

Found 0 matching records in bmi_omr for subject_id 16893343
Found 0 matching records in bmi_omr for subject_id 16901966
Found 23 matching records in bmi_omr for subject_id 16921793
Found 3 matching records in bmi_omr for subject_id 16926271
Found 0 matching records in bmi_omr for subject_id 16931085
Found 1 matching records in bmi_omr for subject_id 16955775
Found 0 matching records in bmi_omr for subject_id 16957926
Found 8 matching records in bmi_omr for subject_id 16967171
Found 0 matching records in bmi_omr for subject_id 16979833
Found 76 matching records in bmi_omr for subject_id 17008145
Found 0 matching records in bmi_omr for subject_id 17014959
Found 0 matching records in bmi_omr for subject_id 17030222
Found 0 matching records in bmi_omr for subject_id 17031760
Found 0 matching records in bmi_omr for subject_id 17059345
Found 0 matching records in bmi_omr for subject_id 17075739
Found 44 matching records in bmi_omr for subject_id 17084051
Found 0 matching records in bmi_omr f

/tmp/ipykernel_1928800/1339896180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/1339896180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/1339896180.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [74]:
target[target['bmi'].isna()]

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi
30,10048899,20315647,30644814,2165-10-20 19:30:00.000000,2165-10-22 16:23:10,1.870255,91,F,WHITE,NaN,64.836828,NaN,NaN
34,10062020,25255239,35323579,2113-05-31 21:18:47.000000,2113-06-01 15:49:43,0.771481,66,F,WHITE,NaN,99.954710,NaN,NaN
43,10094805,21370539,34488364,2182-04-23 19:15:11.000000,2182-04-29 16:59:03,5.905463,74,M,WHITE,NaN,65.835784,NaN,NaN
69,10132888,21293774,34165381,2127-09-24 03:47:00.000000,2127-09-24 22:38:33,0.785799,89,M,ASIAN,NaN,65.862288,66.0,NaN
79,10155871,25868401,38274571,2153-03-29 18:54:00.000000,2153-03-30 17:26:37,0.939317,71,M,WHITE,NaN,103.083262,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4716,19867569,27384522,35392118,2114-03-03 09:38:40.000000,2114-03-05 13:19:50,2.153588,61,F,UNABLE TO OBTAIN,NaN,100.859506,103.6,NaN
4718,19869590,24842662,34062535,2117-02-02 11:30:36.000000,2117-02-05 18:13:28,3.279769,45,M,WHITE,NaN,130.000000,132.0,NaN
4723,19883399,27355975,39506648,2133-07-24 16:32:45.000000,2133-08-20 23:17:46,27.281262,84,M,UNABLE TO OBTAIN,NaN,105.353246,101.5,NaN
4725,19887023,24942188,35949283,2128-11-12 09:55:34.000000,2128-11-13 04:09:54,0.759954,91,M,WHITE,NaN,80.000000,NaN,NaN


In [80]:
def add_bmi_omr(target):
    for index, row in target[target['bmi'].isna()].iterrows():
        records = bmi_omr[bmi_omr['subject_id'] == row['subject_id']]
        print(f"Found {len(records)} matching records in bmi_omr for subject_id {row['subject_id']}")
        
        if not records.empty:
            records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
            closest_bmi_record = records.loc[records['time_diff'].idxmin()]
            target.at[index, 'bmi'] = closest_bmi_record['result_value']
    return target

In [81]:
target_new = add_bmi_omr(target)
target_new[target_new['bmi'].isna()]

Found 0 matching records in bmi_omr for subject_id 10048899
Found 0 matching records in bmi_omr for subject_id 10062020
Found 1 matching records in bmi_omr for subject_id 10078805
Found 0 matching records in bmi_omr for subject_id 10094805
Found 58 matching records in bmi_omr for subject_id 10102822
Found 9 matching records in bmi_omr for subject_id 10104549
Found 0 matching records in bmi_omr for subject_id 10132888
Found 34 matching records in bmi_omr for subject_id 10150056
Found 0 matching records in bmi_omr for subject_id 10155871
Found 0 matching records in bmi_omr for subject_id 10163041
Found 0 matching records in bmi_omr for subject_id 10182211
Found 25 matching records in bmi_omr for subject_id 10221321
Found 0 matching records in bmi_omr for subject_id 10261084
Found 49 matching records in bmi_omr for subject_id 10265482
Found 0 matching records in bmi_omr for subject_id 10282931
Found 1 matching records in bmi_omr for subject_id 10288279
Found 0 matching records in bmi_omr 

/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

Found 10 matching records in bmi_omr for subject_id 13194166
Found 1 matching records in bmi_omr for subject_id 13211467
Found 38 matching records in bmi_omr for subject_id 13214820
Found 2 matching records in bmi_omr for subject_id 13229117
Found 8 matching records in bmi_omr for subject_id 13262421
Found 0 matching records in bmi_omr for subject_id 13266707
Found 0 matching records in bmi_omr for subject_id 13273685
Found 0 matching records in bmi_omr for subject_id 13273685
Found 0 matching records in bmi_omr for subject_id 13281743
Found 0 matching records in bmi_omr for subject_id 13299872
Found 0 matching records in bmi_omr for subject_id 13331522
Found 1 matching records in bmi_omr for subject_id 13364910
Found 65 matching records in bmi_omr for subject_id 13398212
Found 6 matching records in bmi_omr for subject_id 13401524
Found 0 matching records in bmi_omr for subject_id 13421733
Found 0 matching records in bmi_omr for subject_id 13434840
Found 0 matching records in bmi_omr f

/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

Found 1 matching records in bmi_omr for subject_id 16852221
Found 2 matching records in bmi_omr for subject_id 16868333
Found 36 matching records in bmi_omr for subject_id 16878717
Found 2 matching records in bmi_omr for subject_id 16889414
Found 0 matching records in bmi_omr for subject_id 16891384
Found 0 matching records in bmi_omr for subject_id 16893343
Found 0 matching records in bmi_omr for subject_id 16901966
Found 23 matching records in bmi_omr for subject_id 16921793
Found 3 matching records in bmi_omr for subject_id 16926271
Found 0 matching records in bmi_omr for subject_id 16931085
Found 1 matching records in bmi_omr for subject_id 16955775
Found 0 matching records in bmi_omr for subject_id 16957926
Found 8 matching records in bmi_omr for subject_id 16967171
Found 0 matching records in bmi_omr for subject_id 16979833
Found 76 matching records in bmi_omr for subject_id 17008145
Found 0 matching records in bmi_omr for subject_id 17014959
Found 0 matching records in bmi_omr f

/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['time_diff'] = abs(pd.to_datetime(records['chartdate']) - pd.to_datetime(row['intime']))
/tmp/ipykernel_1928800/4089781703.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi
30,10048899,20315647,30644814,2165-10-20 19:30:00.000000,2165-10-22 16:23:10,1.870255,91,F,WHITE,NaN,64.836828,NaN,NaN
34,10062020,25255239,35323579,2113-05-31 21:18:47.000000,2113-06-01 15:49:43,0.771481,66,F,WHITE,NaN,99.954710,NaN,NaN
43,10094805,21370539,34488364,2182-04-23 19:15:11.000000,2182-04-29 16:59:03,5.905463,74,M,WHITE,NaN,65.835784,NaN,NaN
69,10132888,21293774,34165381,2127-09-24 03:47:00.000000,2127-09-24 22:38:33,0.785799,89,M,ASIAN,NaN,65.862288,66.0,NaN
79,10155871,25868401,38274571,2153-03-29 18:54:00.000000,2153-03-30 17:26:37,0.939317,71,M,WHITE,NaN,103.083262,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4716,19867569,27384522,35392118,2114-03-03 09:38:40.000000,2114-03-05 13:19:50,2.153588,61,F,UNABLE TO OBTAIN,NaN,100.859506,103.6,NaN
4718,19869590,24842662,34062535,2117-02-02 11:30:36.000000,2117-02-05 18:13:28,3.279769,45,M,WHITE,NaN,130.000000,132.0,NaN
4723,19883399,27355975,39506648,2133-07-24 16:32:45.000000,2133-08-20 23:17:46,27.281262,84,M,UNABLE TO OBTAIN,NaN,105.353246,101.5,NaN
4725,19887023,24942188,35949283,2128-11-12 09:55:34.000000,2128-11-13 04:09:54,0.759954,91,M,WHITE,NaN,80.000000,NaN,NaN


In [94]:
target_new.dtypes

subject_id            int64
hadm_id               int64
stay_id               int64
intime               object
outtime              object
los                 float64
age_at_admission      int64
gender               object
race                 object
height              float64
weight_admit        float64
weight_first_day    float64
bmi                  object
dtype: object

In [84]:
target_new[target_new['subject_id'] == 19998330]

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi
4776,19998330,23151993,31085771,2178-09-20 21:40:00.000000,2178-09-21 18:06:26,0.851690,72,F,BLACK/AFRICAN AMERICAN,NaN,76.205960,NaN,24.7
4777,19998330,21135114,31417783,2178-10-21 17:20:00.000000,2178-10-23 17:43:28,2.016296,72,F,BLACK/AFRICAN AMERICAN,NaN,77.514506,77.65,25.0
4778,19998330,24492004,32641669,2178-10-01 08:51:00.000000,2178-10-03 23:25:08,2.607037,72,F,BLACK/AFRICAN AMERICAN,NaN,77.100000,74.55,25.0
4779,19998330,24096838,33428243,2178-11-27 22:53:00.000000,2178-11-29 21:29:39,1.942118,72,F,BLACK/AFRICAN AMERICAN,NaN,83.000000,NaN,26.4


In [16]:
outfile = open(os.path.join(path_save, 'finish_bmi.pickle'), 'wb')
pickle.dump(target_new, outfile)
outfile.close()

NameError: name 'target_new' is not defined

In [33]:
finish_bmi = pd.read_csv('./eligible_patients_with_demographics.csv')
finish_bmi[finish_bmi['bmi'].isna()]

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi
30,10048899,20315647,30644814,2165-10-20 19:30:00.000000,2165-10-22 16:23:10,1.870255,91,F,WHITE,NaN,64.836828,NaN,NaN
34,10062020,25255239,35323579,2113-05-31 21:18:47.000000,2113-06-01 15:49:43,0.771481,66,F,WHITE,NaN,99.954710,NaN,NaN
43,10094805,21370539,34488364,2182-04-23 19:15:11.000000,2182-04-29 16:59:03,5.905463,74,M,WHITE,NaN,65.835784,NaN,NaN
69,10132888,21293774,34165381,2127-09-24 03:47:00.000000,2127-09-24 22:38:33,0.785799,89,M,ASIAN,NaN,65.862288,66.0,NaN
79,10155871,25868401,38274571,2153-03-29 18:54:00.000000,2153-03-30 17:26:37,0.939317,71,M,WHITE,NaN,103.083262,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4716,19867569,27384522,35392118,2114-03-03 09:38:40.000000,2114-03-05 13:19:50,2.153588,61,F,UNABLE TO OBTAIN,NaN,100.859506,103.6,NaN
4718,19869590,24842662,34062535,2117-02-02 11:30:36.000000,2117-02-05 18:13:28,3.279769,45,M,WHITE,NaN,130.000000,132.0,NaN
4723,19883399,27355975,39506648,2133-07-24 16:32:45.000000,2133-08-20 23:17:46,27.281262,84,M,UNABLE TO OBTAIN,NaN,105.353246,101.5,NaN
4725,19887023,24942188,35949283,2128-11-12 09:55:34.000000,2128-11-13 04:09:54,0.759954,91,M,WHITE,NaN,80.000000,NaN,NaN


In [93]:
finish_bmi.dtypes

subject_id            int64
hadm_id               int64
stay_id               int64
intime               object
outtime              object
los                 float64
age_at_admission      int64
gender               object
race                 object
height              float64
weight_admit        float64
weight_first_day    float64
bmi                 float64
dtype: object

In [23]:
bmi_null = merged[(merged['bmi'].isna()) & (merged['result_value'].notna())]
patients_null = bmi_null.drop_duplicates(subset = ['subject_id'])
patients_null

,subject_id,bmi,result_value
560,10078805,NaN,35.7
595,10102822,NaN,26.8
685,10104549,NaN,32.8
989,10150056,NaN,31.6
1414,10221321,NaN,46.0
...,...,...,...
60281,19937623,NaN,27.3
60314,19962126,NaN,20.7
60339,19965443,NaN,23.8
60355,19978239,NaN,31.4


In [49]:
gcs_query = """
WITH gcs_final AS (
    SELECT
        tp.subject_id,
        tp.stay_id,
        g.charttime,
        MAX(CASE WHEN g.itemid = 223901 THEN g.valuenum END) AS gcs_motor,
        MAX(CASE WHEN g.itemid = 223900 THEN g.valuenum END) AS gcs_verbal,
        MAX(CASE WHEN g.itemid = 220739 THEN g.valuenum END) AS gcs_eyes,
        ROW_NUMBER() OVER (PARTITION BY tp.stay_id ORDER BY g.charttime ASC) AS gcs_seq,
        COALESCE(MAX(CASE WHEN g.itemid = 223901 THEN g.valuenum END), 0) +
        COALESCE(MAX(CASE WHEN g.itemid = 223900 THEN g.valuenum END), 0) +
        COALESCE(MAX(CASE WHEN g.itemid = 220739 THEN g.valuenum END), 0) AS gcs_total
    FROM target_patients tp
    LEFT JOIN gcs_df g ON tp.stay_id = g.stay_id
        AND g.charttime >= DATETIME(tp.intime, '-6 hours')
    WHERE g.warning IS DISTINCT FROM 1
    GROUP BY tp.subject_id, tp.stay_id, g.charttime
)

SELECT
    tp.*,
    gs.gcs_motor,
    gs.gcs_verbal,
    gs.gcs_eyes,
    gs.gcs_total
FROM target_patients tp
LEFT JOIN gcs_final gs ON tp.stay_id = gs.stay_id AND gs.gcs_seq = 1;

"""

In [21]:
def add_gcs(target_patients, gcs_df):
    return psql.sqldf(gcs_query, locals())

In [50]:
target = add_gcs(finish_bmi, gcs_df)
target

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi,gcs_motor,gcs_verbal,gcs_eyes,gcs_total
0,10002428,23473524,35479615,2156-05-11 14:49:34.000000,2156-05-22 14:16:46,10.977222,81,F,WHITE,1.50,48.400000,49.25,21.51,4.0,1.0,1.0,6.0
1,10003400,20214994,32128372,2137-02-25 23:37:19.000000,2137-03-10 21:29:36,12.911308,75,F,BLACK/AFRICAN AMERICAN,1.66,92.902976,94.70,33.71,1.0,1.0,1.0,3.0
2,10003400,23559586,34577403,2137-08-10 19:54:51.000000,2137-08-13 17:54:54,2.916701,75,F,BLACK/AFRICAN AMERICAN,1.66,NaN,90.50,32.84,5.0,1.0,2.0,8.0
3,10004235,24181354,34100191,2196-02-24 17:07:00.000000,2196-02-29 15:58:02,4.952106,47,M,BLACK/CAPE VERDEAN,1.83,126.823339,130.00,37.87,1.0,1.0,2.0,4.0
4,10004401,23920883,32743332,2144-04-21 20:49:00.000000,2144-05-01 13:53:03,9.711146,85,M,WHITE,1.70,90.596303,NaN,31.35,5.0,5.0,4.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4775,19997293,26366652,36403582,2123-11-17 05:47:01.000000,2123-11-19 17:08:30,2.473252,76,M,WHITE,1.82,172.756341,NaN,52.15,6.0,4.0,4.0,14.0
4776,19998330,23151993,31085771,2178-09-20 21:40:00.000000,2178-09-21 18:06:26,0.851690,72,F,BLACK/AFRICAN AMERICAN,NaN,76.205960,NaN,24.70,6.0,5.0,4.0,15.0
4777,19998330,21135114,31417783,2178-10-21 17:20:00.000000,2178-10-23 17:43:28,2.016296,72,F,BLACK/AFRICAN AMERICAN,NaN,77.514506,77.65,25.00,4.0,1.0,2.0,7.0
4778,19998330,24492004,32641669,2178-10-01 08:51:00.000000,2178-10-03 23:25:08,2.607037,72,F,BLACK/AFRICAN AMERICAN,NaN,77.100000,74.55,25.00,5.0,1.0,1.0,7.0


In [51]:
print(target[target['gcs_total'] == 0.])
(target['gcs_total'] == 0.).sum()

      subject_id   hadm_id   stay_id                      intime  \
258     10509305  27629385  38848838  2171-04-09 09:46:14.000000   
860     11738511  28821534  31222837  2150-06-04 06:14:00.000000   
900     11839107  28925461  39983771  2158-06-05 21:56:35.000000   
971     11960419  24403720  30077305  2140-03-08 11:00:50.000000   
988     11988196  26286088  33299263  2160-01-17 00:37:42.000000   
1267    12606113  29557348  30110577  2137-11-26 14:49:34.000000   
1421    12940327  24634273  36499196  2185-05-25 17:30:51.000000   
1465    13017137  25322154  34279854  2155-03-23 14:02:30.000000   
1486    13067703  22432056  33990872  2153-07-28 06:41:54.000000   
2353    14877188  22811307  38335937  2179-06-10 13:29:00.000000   
2787    15745117  23437079  30540914  2183-11-17 17:41:53.000000   
2897    15937789  20703152  37807781  2155-05-03 23:25:33.000000   
3292    16751019  24056955  33917084  2128-04-16 11:52:09.000000   
3976    18290425  21966107  38423613  2184-07-19

17

In [56]:
outfile = open(os.path.join(path_save, 'finish_gcs.pickle'), 'wb')
pickle.dump(target, outfile)
outfile.close()

In [60]:
target[target['gcs_verbal'].isna()]

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi,gcs_motor,gcs_verbal,gcs_eyes,gcs_total
148,10277901,22457836,35649985,2137-06-10 13:09:09.000000,2137-06-14 18:19:52,4.215775,92,M,OTHER,1.52,42.600000,NaN,18.44,6.0,NaN,4.0,10.0
204,10386233,21983314,33456290,2159-10-22 04:19:12.000000,2159-10-24 02:39:50,1.930995,70,M,OTHER,1.74,113.072830,113.2,37.35,6.0,NaN,3.0,9.0
255,10505233,20036938,33430313,2131-10-18 21:09:00.000000,2131-10-20 17:33:19,1.850220,74,M,WHITE,1.78,103.855333,104.0,32.78,6.0,NaN,4.0,10.0
258,10509305,27629385,38848838,2171-04-09 09:46:14.000000,2171-04-09 17:55:29,0.339757,78,M,WHITE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
319,10643827,29803052,30589359,2154-04-14 06:08:00.000000,2154-04-22 21:39:39,8.646979,75,F,UNABLE TO OBTAIN,1.63,64.932187,NaN,24.44,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4497,19477304,25876676,39223303,2125-12-25 01:00:00.000000,2125-12-25 01:39:09,0.027187,80,M,WHITE - BRAZILIAN,1.75,NaN,NaN,21.70,NaN,NaN,NaN,0.0
4518,19508874,29290320,39415876,2207-01-27 01:46:52.000000,2207-03-04 15:38:12,36.577315,87,F,BLACK/CARIBBEAN ISLAND,1.45,57.000000,57.0,27.11,6.0,NaN,4.0,10.0
4594,19645295,21641527,33179426,2135-04-02 08:30:46.000000,2135-04-07 18:42:54,5.425093,85,F,OTHER,1.63,53.633339,NaN,20.19,5.0,NaN,3.0,8.0
4615,19683695,27751015,37245983,2139-10-22 16:10:31.000000,2139-10-22 19:39:09,0.144884,71,M,WHITE,NaN,67.929057,NaN,21.80,NaN,NaN,NaN,0.0


In [3]:
path_save = './pickle'
infile = open(os.path.join(path_save,'finish_gcs.pickle'), 'rb')
target = pickle.load(infile)
infile.close()

In [69]:
len(target['stay_id'].unique())

4780

In [4]:
target_patients = pd.read_csv('/media/data/huyennm/mimic-iv/notebook/eligible_patients.csv')
target_patients

,subject_id,hadm_id,stay_id,charttime,vent,age_at_admission
0,10001843,26133978,39698942,2134-12-05 19:00:00,1,76
1,10001843,26133978,39698942,2134-12-05 20:00:00,1,76
2,10001843,26133978,39698942,2134-12-05 21:00:00,1,76
3,10001843,26133978,39698942,2134-12-05 21:01:00,1,76
4,10001843,26133978,39698942,2134-12-05 23:00:00,1,76
...,...,...,...,...,...,...
201344,19998591,24349193,31144045,2185-07-17 16:00:00,1,53
201345,19998591,24349193,36794489,2185-07-04 07:50:00,1,53
201346,19998591,24349193,36794489,2185-07-04 12:00:00,1,53
201347,19998591,24349193,36794489,2185-07-04 16:00:00,1,53


In [6]:
ventsettings_query = """
SELECT
  ce.stay_id, ce.charttime,
  MAX(CASE WHEN ce.itemid IS NULL OR ce.value IS NULL THEN 0 
           WHEN ce.itemid = 720 AND ce.value != 'Other/Remarks' THEN 1  
           WHEN ce.itemid = 223848 AND ce.value != 'Other' THEN 1
           WHEN ce.itemid = 223849 THEN 1 
           WHEN ce.itemid = 467 AND ce.value = 'Ventilator' THEN 1 
           WHEN ce.itemid IN (445, 448, 449, 450, 1340, 1486, 1600, 224687, 
                              639, 654, 681, 682, 683, 684, 224685, 224684, 224686, 
                              218, 436, 535, 444, 459, 224697, 224695, 224696, 224746, 
                              224747, 221, 1, 1211, 1655, 2000, 226873, 224738, 224419, 
                              224750, 227187, 543, 5865, 5866, 224707, 224709, 224705, 
                              224706, 60, 437, 505, 506, 686, 220339, 224700, 3459, 
                              501, 502, 503, 224702, 223, 667, 668, 669, 670, 671, 672, 
                              224701) 
           THEN 1 ELSE 0 END) AS MechVent,
  MAX(CASE WHEN ce.itemid = 226732 AND ce.value IN 
           ('Nasal cannula', 'Face tent', 'Aerosol-cool', 'Trach mask', 
            'High flow neb', 'Non-rebreather', 'Venti mask', 'Medium conc mask', 
            'T-piece', 'High flow nasal cannula', 'Ultrasonic neb', 'Vapomist') 
           THEN 1 
           WHEN ce.itemid = 467 AND ce.value IN 
           ('Cannula', 'Nasal Cannula', 'None', 'Face Tent', 'Aerosol-Cool', 
            'Trach Mask', 'Hi Flow Neb', 'Non-Rebreather', 'Venti Mask', 
            'Medium Conc Mask', 'Vapotherm', 'T-Piece', 'Hood', 'Hut', 
            'TranstrachealCat', 'Heated Neb', 'Ultrasonic Neb') 
           THEN 1 ELSE 0 END) AS OxygenTherapy,
  MAX(CASE WHEN ce.itemid IS NULL OR ce.value IS NULL THEN 0 
           WHEN ce.itemid = 640 AND ce.value IN ('Extubated', 'Self Extubation') THEN 1 
           ELSE 0 END) AS Extubated,
  MAX(CASE WHEN ce.itemid IS NULL OR ce.value IS NULL THEN 0 
           WHEN ce.itemid = 640 AND ce.value = 'Self Extubation' THEN 1 
           ELSE 0 END) AS SelfExtubated
FROM dur_mv ce
JOIN target_patients tp ON ce.stay_id = tp.stay_id
WHERE ce.value IS NOT NULL
  AND ce.warning IS DISTINCT FROM 1
  AND ce.itemid IN (720, 223849, 223848, 445, 448, 449, 450, 1340, 1486, 
                    1600, 224687, 639, 654, 681, 682, 683, 684, 224685, 
                    224684, 224686, 218, 436, 535, 444, 224697, 224695, 
                    224696, 224746, 224747, 221, 1, 1211, 1655, 2000, 
                    226873, 224738, 224419, 224750, 227187, 543, 5865, 
                    5866, 224707, 224709, 224705, 224706, 60, 437, 505, 
                    506, 686, 220339, 224700, 3459, 501, 502, 503, 
                    224702, 223, 667, 668, 669, 670, 671, 672, 224701, 
                    640, 468, 469, 470, 471, 227287, 226732, 223834, 467)
GROUP BY ce.stay_id, ce.charttime
UNION
SELECT
  pe.stay_id, pe.starttime AS charttime,
  0 AS MechVent, 0 AS OxygenTherapy,
  1 AS Extubated, CASE WHEN pe.itemid = 225468 THEN 1 ELSE 0 END AS SelfExtubated
FROM procedureevents_mv pe
JOIN target_patients tp ON pe.stay_id = tp.stay_id
WHERE pe.itemid IN (227194, 225468, 225477);
"""

In [7]:
def add_ventsettings(target_patients, dur_mv, procedureevents_mv):
    return psql.sqldf(ventsettings_query, locals())

In [8]:
new_target = add_ventsettings(target_patients, dur_mv, procedureevents_mv)

PandaSQLException: (sqlite3.OperationalError) no such table: target
[SQL: 
SELECT
  ce.stay_id, ce.charttime,
  MAX(CASE WHEN ce.itemid IS NULL OR ce.value IS NULL THEN 0 
           WHEN ce.itemid = 720 AND ce.value != 'Other/Remarks' THEN 1  
           WHEN ce.itemid = 223848 AND ce.value != 'Other' THEN 1
           WHEN ce.itemid = 223849 THEN 1 
           WHEN ce.itemid = 467 AND ce.value = 'Ventilator' THEN 1 
           WHEN ce.itemid IN (445, 448, 449, 450, 1340, 1486, 1600, 224687, 
                              639, 654, 681, 682, 683, 684, 224685, 224684, 224686, 
                              218, 436, 535, 444, 459, 224697, 224695, 224696, 224746, 
                              224747, 221, 1, 1211, 1655, 2000, 226873, 224738, 224419, 
                              224750, 227187, 543, 5865, 5866, 224707, 224709, 224705, 
                              224706, 60, 437, 505, 506, 686, 220339, 224700, 3459, 
                              501, 502, 503, 224702, 223, 667, 668

In [99]:
new_target.drop_duplicates(subset='stay_id')

,stay_id,charttime,MechVent,OxygenTherapy,Extubated,SelfExtubated
0,30000213,2162-06-21 05:00:00,1,0,0,0
17,30004391,2153-09-05 13:25:00,1,0,0,0
65,30007565,2141-07-09 01:00:00,1,0,0,0
224,30007983,2176-07-24 04:00:00,1,0,0,0
238,30009597,2175-09-30 22:00:00,1,0,0,0
...,...,...,...,...,...,...
262016,39991569,2179-04-10 17:22:00,1,0,0,0
262032,39992578,2187-09-14 21:00:00,1,0,0,0
262054,39995735,2124-08-15 18:00:00,1,0,0,0
262120,39996123,2134-07-22 02:47:00,0,1,0,0


In [115]:
new_target.drop_duplicates(subset='stay_id')
not_in_new_target = target[~target['stay_id'].isin(new_target['stay_id'].unique())]
list_not_in_nt = not_in_new_target['stay_id'].tolist()
len(not_in_new_target.drop_duplicates(subset='stay_id'))

43

In [117]:
missing_in_dur_mv = dur_mv[dur_mv['stay_id'].isin(list_not_in_nt)]
len(missing_in_dur_mv.drop_duplicates(subset='stay_id'))

35

-> 8 stay_id don't have information in dur_mv (so why tf they are in this list)

In [106]:
in_dur_mv[~in_dur_mv['value'].isna()]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
3654087,14525616,26794444,38487968,70829.0,2175-08-24 01:00:00,2175-08-24 01:08:00,223834,15,15.0,L/min,1.0
3654088,14525616,26794444,38487968,70829.0,2175-08-24 01:00:00,2175-08-24 01:08:00,226732,Bipap mask,NaN,NaN,1.0
5293240,16442703,26707548,34725385,8388.0,2145-10-22 00:00:00,2145-10-22 00:43:00,223834,2,2.0,L/min,1.0
5293241,16442703,26707548,34725385,8388.0,2145-10-22 00:00:00,2145-10-22 00:43:00,226732,Nasal cannula,NaN,NaN,1.0


-> 43 stay_id values have a NULL value or a warning of 1.0, so they are eliminated

In [47]:
ventdurations_query = """
WITH vd0 AS
(
  SELECT
    vs.stay_id,
    CASE WHEN vs.MechVent = 1 THEN LAG(vs.charttime, 1) OVER (PARTITION BY vs.stay_id ORDER BY vs.charttime)
    ELSE NULL END AS charttime_lag,
    vs.charttime, vs.MechVent, vs.OxygenTherapy, vs.Extubated, vs.SelfExtubated
  FROM ventsettings vs
  JOIN target tp ON vs.stay_id = tp.stay_id
),
vd1 AS
(
  SELECT
    vd0.stay_id, vd0.charttime_lag, vd0.charttime, vd0.MechVent,
    vd0.OxygenTherapy, vd0.Extubated, vd0.SelfExtubated,
    CASE WHEN vd0.MechVent = 1 THEN vd0.charttime - vd0.charttime_lag ELSE NULL END AS ventduration,
    LAG(vd0.Extubated, 1) OVER (PARTITION BY vd0.stay_id, 
      CASE WHEN vd0.MechVent = 1 OR vd0.Extubated = 1 THEN 1 ELSE 0 END 
      ORDER BY vd0.charttime) AS ExtubatedLag,
    CASE
      WHEN LAG(vd0.Extubated, 1) OVER (PARTITION BY vd0.stay_id, 
          CASE WHEN vd0.MechVent = 1 OR vd0.Extubated = 1 THEN 1 ELSE 0 END 
          ORDER BY vd0.charttime) = 1 THEN 1
      WHEN vd0.MechVent = 0 AND vd0.OxygenTherapy = 1 THEN 1
      WHEN (vd0.charttime - vd0.charttime_lag) > 8 * 3600 THEN 1
      ELSE 0 END AS newvent
  FROM vd0
),
vd2 AS
(
  SELECT vd1.*, 
    CASE 
      WHEN vd1.MechVent = 1 OR vd1.Extubated = 1 THEN 
        SUM(vd1.newvent) OVER (PARTITION BY vd1.stay_id ORDER BY vd1.charttime) 
      ELSE NULL 
    END AS ventnum
  FROM vd1
)
SELECT vd2.stay_id, DATE(vd2.charttime) AS charttime_day,
  ROW_NUMBER() OVER (PARTITION BY vd2.stay_id ORDER BY vd2.ventnum) AS ventnum,
  MIN(vd2.charttime) AS starttime, MAX(vd2.charttime) AS endtime,
  (strftime('%s', MAX(vd2.charttime)) - strftime('%s', MIN(vd2.charttime))) / 60.0 AS duration_minutes,
  vd2.MechVent, vd2.OxygenTherapy, vd2.Extubated, vd2.SelfExtubated
FROM vd2
JOIN target tp ON vd2.stay_id = tp.stay_id
WHERE vd2.ventnum IS NOT NULL
  AND vd2.MechVent = 1
GROUP BY vd2.stay_id, ventnum, vd2.MechVent, vd2.OxygenTherapy, vd2.Extubated, vd2.SelfExtubated;
"""

In [50]:
def add_ventdurations(new_target, target, dur_mv, procedureevents_mv):
    return psql.sqldf(ventdurations_query, locals())

In [51]:
new_new_target = add_ventdurations(new_target, target, dur_mv, procedureevents_mv)

In [123]:
new_new_target.drop_duplicates(subset='stay_id')

,stay_id,charttime_day,ventnum,starttime,endtime,duration_minutes,MechVent,OxygenTherapy,Extubated,SelfExtubated
0,30000213,2162-06-22,1,2162-06-21 05:00:00,2162-06-22 07:00:00,1560.0,1,0,0,0
1,30004391,2153-09-07,1,2153-09-05 13:25:00,2153-09-07 12:00:00,2795.0,1,0,0,0
3,30007565,2141-07-15,1,2141-07-09 01:00:00,2141-07-15 08:00:00,9060.0,1,0,0,0
6,30007983,2176-07-24,1,2176-07-24 04:00:00,2176-07-24 21:00:00,1020.0,1,0,0,0
7,30009597,2175-10-01,1,2175-09-30 22:00:00,2175-10-01 11:00:00,780.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
6368,39991569,2179-04-11,1,2179-04-10 17:22:00,2179-04-11 16:00:00,1358.0,1,0,0,0
6369,39992578,2187-09-16,1,2187-09-14 21:00:00,2187-09-16 12:00:00,2340.0,1,0,0,0
6370,39995735,2124-08-21,1,2124-08-15 18:00:00,2124-08-21 06:00:00,7920.0,1,0,0,0
6372,39996123,2134-07-22,1,2134-07-22 12:05:00,2134-07-22 20:00:00,475.0,1,0,0,0


In [68]:
list_dont_have_dur = target[~target['stay_id'].isin(new_new_target['stay_id'])]
list_dont_have_dur

,subject_id,hadm_id,stay_id,intime,outtime,los,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi,gcs_motor,gcs_verbal,gcs_eyes,gcs_total
30,10048899,20315647,30644814,2165-10-20 19:30:00.000000,2165-10-22 16:23:10,1.870255,91,F,WHITE,NaN,64.836828,NaN,NaN,6.0,4.0,4.0,14.0
36,10076263,25634900,31003445,2196-05-29 05:59:00.000000,2196-05-31 18:15:50,2.511690,50,F,BLACK/AFRICAN AMERICAN,1.63,52.390456,NaN,19.72,6.0,5.0,4.0,15.0
48,10098215,20652197,36567846,2118-08-14 22:06:03.000000,2118-08-16 21:57:17,1.993912,57,F,WHITE,1.56,51.850390,51.9,21.31,6.0,5.0,4.0,15.0
68,10132611,28996766,31179345,2168-02-22 05:05:00.000000,2168-02-24 16:36:01,2.479873,74,M,WHITE,1.78,86.300000,NaN,27.24,6.0,5.0,4.0,15.0
69,10132888,21293774,34165381,2127-09-24 03:47:00.000000,2127-09-24 22:38:33,0.785799,89,M,ASIAN,NaN,65.862288,66.0,NaN,5.0,4.0,3.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4746,19952398,24995993,32679065,2127-04-09 19:08:03.000000,2127-04-17 20:46:19,8.068241,70,M,WHITE,1.78,50.873628,NaN,16.06,6.0,5.0,4.0,15.0
4762,19978239,21674747,38949980,2120-12-02 18:56:20.000000,2120-12-06 21:11:51,4.094109,82,F,WHITE,NaN,101.000000,101.0,31.40,6.0,5.0,4.0,15.0
4765,19983257,21141313,37843321,2166-01-09 15:19:16.000000,2166-01-15 21:27:49,6.255937,64,M,WHITE,1.65,140.000000,NaN,51.42,6.0,5.0,4.0,15.0
4775,19997293,26366652,36403582,2123-11-17 05:47:01.000000,2123-11-19 17:08:30,2.473252,76,M,WHITE,1.82,172.756341,NaN,52.15,6.0,4.0,4.0,14.0


In [65]:
test_list = list_dont_have_dur['stay_id'].tolist()
test_list

[30644814,
 31003445,
 36567846,
 31179345,
 34165381,
 36538238,
 32831963,
 30337986,
 37496349,
 35474807,
 37580514,
 38065034,
 39171509,
 37931604,
 35031000,
 34436779,
 35269020,
 30800478,
 31012664,
 37537235,
 32068207,
 30770361,
 32165451,
 33215882,
 34135108,
 38154150,
 39267426,
 32652263,
 32815100,
 34206944,
 39431744,
 38046850,
 33345883,
 32428867,
 31698532,
 30649699,
 30382119,
 34662305,
 35001991,
 36301525,
 37741300,
 33155340,
 33652366,
 34577954,
 33840497,
 31865868,
 33279342,
 39079041,
 32642728,
 30713595,
 31951080,
 34033580,
 36088280,
 34673429,
 34300466,
 35810878,
 31681305,
 37079354,
 35352693,
 37792645,
 31597960,
 34439141,
 35895937,
 32640656,
 38528141,
 30978046,
 37530970,
 39699085,
 32367149,
 39158918,
 33891173,
 33039123,
 36282564,
 36453823,
 33577025,
 32721770,
 37854016,
 39983771,
 30011071,
 32969864,
 34633573,
 39945335,
 37883545,
 31707457,
 31141250,
 30077305,
 39220558,
 30513900,
 38294786,
 38845095,
 37667136,

In [66]:
list_patient_not_have_duration = []
for i in test_list:
    test = dur_mv[dur_mv['stay_id'] == i]
    list_patient_not_have_duration.append(test)

list_patients_not_have_duration = pd.concat(list_patient_not_have_duration, axis=0)

In [67]:
list_patients_not_have_duration

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
43662,10048899,20315647,30644814,18428.0,2165-10-21 08:00:00,2165-10-21 07:57:00,223834,3,3.0,L/min,0.0
43663,10048899,20315647,30644814,18428.0,2165-10-21 08:00:00,2165-10-21 07:57:00,226732,Nasal cannula,NaN,NaN,0.0
43664,10048899,20315647,30644814,18428.0,2165-10-21 11:00:00,2165-10-21 11:09:00,226732,Nasal cannula,NaN,NaN,0.0
43665,10048899,20315647,30644814,18428.0,2165-10-21 11:00:00,2165-10-21 11:10:00,223834,2,2.0,L/min,1.0
43666,10048899,20315647,30644814,18428.0,2165-10-21 12:00:00,2165-10-21 15:54:00,226732,Nasal cannula,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8159235,19998330,23151993,31085771,68320.0,2178-09-21 12:00:00,2178-09-21 14:23:00,226732,Nasal cannula,NaN,NaN,0.0
8159236,19998330,23151993,31085771,68320.0,2178-09-21 16:00:00,2178-09-21 16:11:00,223834,3,3.0,L/min,0.0
8159237,19998330,23151993,31085771,68320.0,2178-09-21 16:00:00,2178-09-21 16:11:00,226732,Nasal cannula,NaN,NaN,0.0
8159238,19998330,23151993,31085771,72599.0,2178-09-21 04:00:00,2178-09-21 04:07:00,223834,8,8.0,L/min,0.0


In [7]:
height_df = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/height.csv')
weight_df = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/weight.csv')

/tmp/ipykernel_1239107/167791002.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  weight_df = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/weight.csv')


In [14]:
demo = pd.read_csv('/media/data/huyennm/mimic-iv/notebook/demographics/eligible_patients_with_demographics_gcs.csv')

In [17]:
demo.drop_duplicates(subset='subject_id')

,subject_id,hadm_id,stay_id,charttime,vent,age_at_admission,gender,race,height,weight_admit,weight_first_day,bmi,gcs_motor,gcs_verbal,gcs_eyes,gcs_total
0,10000690,25860671,37081114,2150-11-02 19:40:00,1,86,F,WHITE,NaN,55.185919,NaN,23.00,6.0,5.0,4.0,15.0
1,10000980,26913865,39765666,2189-06-27 09:00:00,1,76,F,BLACK/AFRICAN AMERICAN,NaN,57.061431,NaN,30.60,6.0,5.0,4.0,15.0
2,10001843,26133978,39698942,2134-12-05 19:00:00,1,76,M,WHITE,NaN,76.728949,NaN,30.60,6.0,4.0,3.0,13.0
3,10002013,23581541,39060235,2160-05-18 14:00:00,1,57,F,OTHER,1.57,95.899846,104.1,38.91,1.0,1.0,1.0,3.0
4,10002155,28994087,31090461,2130-09-24 01:00:00,1,82,F,WHITE,NaN,47.933231,NaN,20.30,6.0,5.0,4.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25769,19997473,27787494,32134105,2173-09-17 15:42:00,1,82,F,WHITE,1.57,62.959267,65.5,25.54,6.0,5.0,4.0,15.0
25771,19997752,29452285,34531437,2128-03-02 13:00:00,1,66,F,WHITE,1.70,83.884544,87.8,29.03,1.0,1.0,1.0,3.0
25772,19997760,25252564,33057362,2190-09-09 23:00:00,1,53,M,BLACK/AFRICAN AMERICAN,1.85,125.000000,133.5,36.52,4.0,1.0,2.0,7.0
25775,19998330,23151993,31085771,2178-09-20 21:00:00,1,72,F,BLACK/AFRICAN AMERICAN,NaN,76.205960,NaN,24.70,6.0,5.0,4.0,15.0
